### Etymology

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import re
import requests
from urllib.request import Request, urlopen
from urllib import parse
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/kurubal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/kurubal/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
lang_folder = "Turkish"

In [13]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Result/Word/Merge/Word_Merge2.xlsx")
#df_word = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Result/Word/Merge/Word_Merge2.csv")
df_word = df_word.head(10000)
df_word

,word,frequency
0,que,37853284
1,de,37809537
2,no,33043466
3,a,25439588
4,la,24024343
...,...,...
9995,tácticas,6193
9996,dobles,6191
9997,ollie,6190
9998,gerald,6190


In [64]:
#word_lemma = []
#for word in df_word["word"]:
#    word_lemma.append(WordNetLemmatizer().lemmatize(word))
#    word_unique = set(word_lemma)

In [65]:
#len(word_lemma)

In [66]:
#len(word_unique)

In [14]:
word_list = df_word["word"].to_list()

In [15]:
result_list = []
#for i in word_unique:
for i in word_list:
    try:
        response = requests.get(f"https://etymologeek.com/search/all/{i}")
        soup = BeautifulSoup(response.content, 'lxml')
        word = soup.find('tbody', attrs={"id":"tb"})
        dict_word = word.findAll('td', attrs={"data-th":"Dic. entry"})
        lang = word.findAll('td', attrs={"data-th":"Language"})
        definition = word.findAll('td', attrs={"data-th":"Definition"})
        a_href = word.findAll('td', attrs={"data-th":"Dic. entry"})
        iter_values = zip(dict_word, lang, definition, a_href)
        for a, b, c, d in iter_values:
            var1 = a.get_text()
            var2 = b.get_text()  # string
            var3 = c.get_text()  # string
            var4 = d.find('a').get('href')
            var5 = f"https://etymologeek.com{var4}"
            result_list.append([i, var1, var2, var3, var5])
    except:
        pass

df_result = pd.DataFrame(result_list)
df_result.rename(columns={0:"search_word", 1:"dict_entry", 2:"language", 3:"definition", 4:"href"}, inplace=True)
df_result

In [11]:
df_result.to_excel(f"{lang_folder.lower().capitalize()}_Etymologeek_Main_Result.xlsx", index=False)

#### Scrapping Additional Info

In [38]:
df_main_word = pd.read_excel(f"{lang_folder.lower().capitalize()}_Etymologeek_Main_Result.xlsx")
df_main_word

,search_word,dict_entry,language,definition,href
0,لا,لا,Arabic,"(Conjunction Prefix Particle) Not, and not Non...",https://etymologeek.com/ara/لا
1,لا,لا,Persian,"(Noun) (music) A, La (the sixth note of the fi...",https://etymologeek.com/fas/لا
2,لا,لا,Gurani,NaN,https://etymologeek.com/hac/لا
3,لا,لاءوتن,Malay,(Noun) Ocean (one of the five large bodies of ...,https://etymologeek.com/msa/لاءوتن
4,لا,لابھ,Urdu,(Noun) Advantage. Gain. Profit.Advantage. Gain...,https://etymologeek.com/urd/لابھ
...,...,...,...,...,...
4078,العربية,العربية,Arabic,(Proper noun) The Arabic language.The Arabic l...,https://etymologeek.com/ara/العربية
4079,إلين,إلين,Hijazi Arabic,(Preposition) Until.Until.,https://etymologeek.com/acw/إلين
4080,إلين,إلينا,Arabic,(Adverb) To us.To us.,https://etymologeek.com/ara/إلينا
4081,مستقر,مستقر,Arabic,"(Noun) Fixed domicile, fixed residence, fixed ...",https://etymologeek.com/ara/مستقر


In [39]:
df_main_word.search_word.nunique()

1415

In [40]:
df_main_word.language.nunique()

62

In [41]:
search_word = df_main_word["search_word"].to_list()
dict_entry_main = df_main_word["dict_entry"].to_list()
language_main = df_main_word["language"].to_list()
definition_main = df_main_word["definition"].to_list()	
href_url = df_main_word["href"].to_list()
main_iter_values = zip(search_word, dict_entry_main, language_main, definition_main, href_url)
result_list2 = []
for search, dict_ent, lang_main, def_main, href in main_iter_values:
    try:
        response = requests.get(f"{href}")
        soup = BeautifulSoup(response.content, 'lxml')
        word = soup.find('tbody', attrs={"id":"tb"})
        dict_entry = word.findAll('td', attrs={"data-th":"Dic. entry"})
        lang = word.findAll('td', attrs={"data-th":"Language"})
        definition = word.findAll('td', attrs={"data-th":"Definition"})
        iter_values = zip(dict_entry, lang, definition)
        for a, b, c in iter_values:
            var1 = a.get_text()
            var2 = b.get_text()
            var3 = c.get_text()
            result_list2.append([search, dict_ent, lang_main, def_main, var1, var2, var3])
    except:
        pass

df_result2 = pd.DataFrame(result_list2)
df_result2.rename(columns={0:"search_word", 1:"dict_entry_main", 2:"language_main", 3:"definition_main", 4:"dict_entry", 5:"language", 6:"definition"}, inplace=True)
df_result2

,search_word,dict_entry_main,language_main,definition_main,dict_entry,language,definition
0,لا,لا,Arabic,"(Conjunction Prefix Particle) Not, and not Non...",ليس,Arabic (ara),(copulative) not to be. Not to exist; (third p...
1,لا,لا,Arabic,"(Conjunction Prefix Particle) Not, and not Non...",*lā,Proto-Semitic (sem-pro),No Not.
2,لا,لا,Arabic,"(Conjunction Prefix Particle) Not, and not Non...",لا,Arabic (ara),"Not, and not Non-, un-, in- Do not; expresses ..."
3,لا,لا,Persian,"(Noun) (music) A, La (the sixth note of the fi...",لا,Persian,"(Noun) (music) A, La (the sixth note of the fi..."
4,لا,لا,Persian,"(Noun) (music) A, La (the sixth note of the fi...",لا,Persian,(Particle) (rare) un-.(rare) un-.
...,...,...,...,...,...,...,...
12031,إلين,إلينا,Arabic,(Adverb) To us.To us.,إلينا,Arabic (ara),To us.
12032,مستقر,مستقر,Arabic,"(Noun) Fixed domicile, fixed residence, fixed ...",مستقر,Arabic,"(Adjective) Confessing. Firm, solid, stable. F..."
12033,مستقر,مستقر,Arabic,"(Noun) Fixed domicile, fixed residence, fixed ...",مستقر,Arabic,"(Noun) Fixed domicile, fixed residence, fixed ..."
12034,صيني,صينية,Arabic,(Noun) (usually definite) Chinese language.(us...,لُغَة صِينِيَّة,Arabic (ara),


In [42]:
df_result2.language_main = df_result2.language_main.apply(lambda x: x.strip())
df_result2.language = df_result2.language.apply(lambda x: x.strip())

In [45]:
df_result2.to_excel(f"{lang_folder.lower().capitalize()}_Etymologeek_Main_And_Additional_Result.xlsx", index=False)

#### Language Pair

In [3]:
lang_folder = "Turkish"

In [4]:
df_result2 = pd.read_excel(f"{lang_folder.lower().capitalize()}_Etymologeek_Main_And_Additional_Result.xlsx")
df_result2

,search_word,dict_entry_main,language_main,definition_main,dict_entry,language,definition
0,bir,bir,Crimean Tatar,(Numeral) (cardinal) one.(cardinal) one.,*bīr,Proto-Turkic (trk-pro),One.
1,bir,bir,Crimean Tatar,(Numeral) (cardinal) one.(cardinal) one.,bir,Crimean Tatar (crh),(cardinal) one.
2,bir,bir,Middle English,(Noun) A charge on an account.. A strong or fa...,*buriz,Proto-Germanic (gem-pro),Male offspring; son.
3,bir,bir,Middle English,(Noun) A charge on an account.. A strong or fa...,*bʰrewh₁-,Proto-Indo-European (ine-pro),NaN
4,bir,bir,Middle English,(Noun) A charge on an account.. A strong or fa...,*buriz,Proto-Germanic (gem-pro),Elevation; hill.
...,...,...,...,...,...,...,...
54274,şah,şahin,Turkish,(Noun) Buteo.Buteo.,şahin,Turkish (tur),Buteo.
54275,şah,şahıs,Turkish,(Noun) Person.Person.,شخص,Arabic (ara),"Figure, character. Person, individual. Someone..."
54276,şah,şahıs,Turkish,(Noun) Person.Person.,şahıs,Turkish (tur),Person.
54277,şah,şahsi,Turkish,(Adjective) Personal.Personal.,شخصی,Persian (fas),Personal.


In [ ]:
df_result2.language_main.unique()

In [ ]:
df_result2.language.unique()

In [48]:
lang_pair = "Turkish (tur)"

In [49]:
df_lang_pair = df_result2[(df_result2["language_main"] == f"{lang_folder.lower().capitalize()}") & (df_result2["language"] == f"{lang_pair.lower().capitalize()}")]
df_lang_pair

,search_word,dict_entry_main,language_main,definition_main,dict_entry,language,definition
1628,الأن,الأناضول,Arabic,(Proper noun) Anatolia.Anatolia.,Anadolu,Turkish (tur),Anatolia.
1744,دون,دونم,Arabic,(Noun) Dunam.Dunam.,dönmek,Turkish (tur),To be converted to another religion. To return...
3929,شعر,شعر,Arabic,"(Noun Noun Noun Verb) Knowing, knowledge. Noti...",شعار,Turkish (tur),NaN
6520,كافي,كافيار,Arabic,(Noun) Caviar.Caviar.,havyar,Turkish (tur),Caviar.
10555,كاف,كافيار,Arabic,(Noun) Caviar.Caviar.,havyar,Turkish (tur),Caviar.


In [ ]:
df_lang_pair.to_excel(f"{lang_folder.lower().capitalize()}_And_{lang_pair.lower().capitalize()}_Etymologeek_Result.xlsx", index=False)